# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 8:30AM,258092,10,0086421850,ISDIN Corporation,Released
1,Feb 24 2023 8:30AM,258092,10,0086421852,ISDIN Corporation,Released
2,Feb 24 2023 8:30AM,258092,10,0086421853,ISDIN Corporation,Released
3,Feb 24 2023 8:30AM,258092,10,0086421830,ISDIN Corporation,Released
4,Feb 24 2023 8:30AM,258092,10,0086421836,ISDIN Corporation,Released
5,Feb 24 2023 8:30AM,258092,10,0086421835,ISDIN Corporation,Released
6,Feb 24 2023 8:30AM,258092,10,0086421837,ISDIN Corporation,Released
7,Feb 24 2023 8:30AM,258092,10,0086421851,ISDIN Corporation,Released
8,Feb 24 2023 8:30AM,258092,10,0086421849,ISDIN Corporation,Released
9,Feb 24 2023 8:30AM,258092,10,0086421820,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,258091,Released,4
39,258092,Released,56
40,258093,Released,1
41,258094,Released,1
42,258095,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258091,NaN,NaN,4.0
258092,NaN,NaN,56.0
258093,NaN,NaN,1.0
258094,NaN,NaN,1.0
258095,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0.0,0.0,13.0
257943,0.0,0.0,1.0
257983,0.0,0.0,2.0
257986,0.0,0.0,2.0
257988,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0,0,13
257943,0,0,1
257983,0,0,2
257986,0,0,2
257988,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,0,0,13
1,257943,0,0,1
2,257983,0,0,2
3,257986,0,0,2
4,257988,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,,,13
1,257943,,,1
2,257983,,,2
3,257986,,,2
4,257988,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation
51,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation
57,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation
111,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc."
116,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc."
122,Feb 24 2023 8:30AM,258088,10,"Virtue Rx, LLC"
123,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc."
124,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc."
125,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences
126,Feb 23 2023 2:40PM,258084,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation,,,56
1,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,,,4
2,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,,,56
3,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",,,5
4,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc.",,,1
5,Feb 24 2023 8:30AM,258088,10,"Virtue Rx, LLC",,,1
6,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",,,1
7,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",,,1
8,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences,,,1
9,Feb 23 2023 2:40PM,258084,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation,56,,
1,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
2,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
3,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
4,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc.",1,,
5,Feb 24 2023 8:30AM,258088,10,"Virtue Rx, LLC",1,,
6,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",1,,
7,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",1,,
8,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences,1,,
9,Feb 23 2023 2:40PM,258084,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation,56,,
1,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
2,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
3,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
4,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation,56.0,NaN,NaN
1,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,NaN,NaN
2,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,NaN,NaN
3,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,NaN,NaN
4,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258092,10,ISDIN Corporation,56.0,0.0,0.0
1,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,0.0,0.0
2,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,0.0,0.0
3,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,0.0,0.0
4,Feb 24 2023 8:30AM,258093,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6451441,258.0,14.0,22.0
12,258044,1.0,0.0,0.0
15,1031800,18.0,0.0,0.0
19,1290402,35.0,1.0,0.0
20,258052,1.0,0.0,0.0
21,258000,1.0,0.0,0.0
22,258095,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6451441,258.0,14.0,22.0
1,12,258044,1.0,0.0,0.0
2,15,1031800,18.0,0.0,0.0
3,19,1290402,35.0,1.0,0.0
4,20,258052,1.0,0.0,0.0
5,21,258000,1.0,0.0,0.0
6,22,258095,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,258.0,14.0,22.0
1,12,1.0,0.0,0.0
2,15,18.0,0.0,0.0
3,19,35.0,1.0,0.0
4,20,1.0,0.0,0.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,258.0
1,12,Released,1.0
2,15,Released,18.0
3,19,Released,35.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0
Released,258.0,1.0,18.0,35.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Released,258.0,1.0,18.0,35.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Released,258.0,1.0,18.0,35.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()